# Import libs

In [ ]:
from google.colab import drive

drive.mount('/content/drive')
path = '/content/drive/MyDrive/TTKS 20203/VIB/Track 1 Datarathon/'
%cd $path
!ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/TTKS 20203/VIB/Track 1 Datarathon
'0.Data VIB Hackathon Guidline.xlsx'   4.Data_Deposit.csv
 1.Data_Customer.csv		       5.Data_Lending.csv
 2.Data_MyVIB_Transaction.csv	       6.Data_Card.csv
 3.Data_MyVIB_Activity.csv	       label_times.csv
 3.Data_MyVIB_Activity_New


In [ ]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

df_3 = pd.read_csv(path + '3.Data_MyVIB_Activity.csv',
                        parse_dates=['ACTIVITY_DATE'],
                        infer_datetime_format = True,
                        dtype = {'DAY_OF_WEEK': 'category', 'ACTIVITY_HOUR': 'category', 'ACTIVITY_NAME': 'category'})
df_3.head()

,ACTIVITY_DATE,DAY_OF_WEEK,ACTIVITY_HOUR,ACTIVITY_NO,CUSTOMER_NUMBER,ACTIVITY_NAME
0,2019-01-22,Tue,20,2,630178,MB_INTEREST_RATE_VIEW
1,2019-01-10,Thu,18,16,630178,MB_INTEREST_RATE_VIEW
2,2019-01-26,Sat,17,15,630178,MB_INTEREST_RATE_VIEW
3,2019-01-02,Wed,9,27,630178,MB_INTEREST_RATE_VIEW
4,2019-01-19,Sat,8,3,630178,MB_INTEREST_RATE_VIEW


In [ ]:
df_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16132675 entries, 0 to 16132674
Data columns (total 6 columns):
 #   Column           Dtype         
---  ------           -----         
 0   ACTIVITY_DATE    datetime64[ns]
 1   DAY_OF_WEEK      category      
 2   ACTIVITY_HOUR    category      
 3   ACTIVITY_NO      int64         
 4   CUSTOMER_NUMBER  int64         
 5   ACTIVITY_NAME    category      
dtypes: category(3), datetime64[ns](1), int64(2)
memory usage: 415.4 MB


In [ ]:
df_3['ACTIVITY_NAME'].value_counts()

LOGIN                            3638224
LOGOUT                           3499251
QUERY_ACCOUNT_INFORMATION        2566128
MB_INTEREST_RATE_VIEW            2448877
TRANSACTION_OVERVIEW_QUERY       1079162
LOGIN_FINGER                      784355
TRANSFER_VIA_SML                  679219
LOGIN_FACEID                      237933
TRANSFER_VIB_ACCOUNT              216639
TOPUP_MOBILE                      206441
MB_LOCATION_POS_VIEW              184072
MB_ACCOUNT_QUICK_BALANCE          170850
AUTHENTICATION                    144475
MB_SET_PIN                         69744
MB_BILLPAY                         34703
TRANSFER_VIA_PAYMENT_CENTER        31443
MB_LOCATION_BRANCH_VIEW            28239
CHANGE_PASSWORD                    26410
TRANSFER_VIA_SML_ACCOUNT           20358
MB_LOCATION_ATM_VIEW               17996
TRANSACTION_DETAIL_QUERY           15658
MB_CHANGE_PIN                       8028
MB_RESET_PIN                        7496
MB_EXCHANGE_RATE_VIEW               5618
QUERY_CURRENT_AC

# Preprocessing

In [ ]:
cols = ['ACTIVITY_DATE','CUSTOMER_NUMBER']
cols1 = ['ACTIVITY_HOUR','ACTIVITY_NO','ACTIVITY_NAME']

df3 = df_3.groupby(cols)[cols1].agg(list).reset_index()

In [ ]:
df3

#Predict Engineering

In [ ]:
def label_customer(customer_id, customer_activities, prediction_date, churn_days, 
                   lead_time = 1, prediction_window = 1, return_act = False):

    activities = customer_activities.copy()
    
    activities['ACTIVITY_DATE'] = pd.to_datetime(activities['ACTIVITY_DATE'])
    activities.sort_values(['ACTIVITY_DATE'], inplace = True)

    activities['NEXT_ACTIVITY_DATE'] = activities['ACTIVITY_DATE'].shift(-1)

    activities['DIFFERENCE_DAYS'] = (activities['NEXT_ACTIVITY_DATE'] - activities['ACTIVITY_DATE']).dt.days
    
    activities['CHURN'] = activities['DIFFERENCE_DAYS'] > churn_days
    
    activities.loc[activities['CHURN'] == True, 'CHURN_DATE'] = activities.loc[activities['CHURN'] == True, 
                                                      'ACTIVITY_DATE'] + pd.Timedelta(churn_days + 1, 'd')
    
    first_activity = activities['ACTIVITY_DATE'].min()
    last_activity = activities['ACTIVITY_DATE'].max()
    start_date = pd.datetime(first_activity.year, first_activity.month, 1)
    
    if last_activity.month == 12:
        end_date = pd.datetime(last_activity.year + 1, 1, 1)
    else:
        end_date = pd.datetime(last_activity.year, last_activity.month + 1, 1)
    
    label_times = pd.DataFrame({'CUTOFF_TIME': pd.date_range(start_date, end_date, freq = prediction_date),
                                'CUSTOMER_NUMBER': customer_id})
    
    label_times['PREDICTION_WINDOW_START'] = label_times['CUTOFF_TIME'].shift(-lead_time)
    label_times['PREDICTION_WINDOW_END'] = label_times['CUTOFF_TIME'].shift(-(lead_time + prediction_window))
    
    previous_churn_date = None

    for i, row in label_times.iterrows():
        
        churn_date = pd.NaT
        label = np.nan

        window_start = row['PREDICTION_WINDOW_START']
        window_end = row['PREDICTION_WINDOW_END']

        churns = activities.loc[(activities['CHURN_DATE'] >= window_start) & 
                                  (activities['CHURN_DATE'] < window_end), 'CHURN_DATE']

        if not churns.empty:
            label = 1 # churn
            churn_date = churns.values[0]

            if not previous_churn_date:
                before_idx = label_times.loc[(label_times['CUTOFF_TIME'] <= churn_date)].index
            else:
                before_idx = label_times.loc[(label_times['CUTOFF_TIME'] <= churn_date) & 
                                             (label_times['CUTOFF_TIME'] > previous_churn_date)].index

            label_times.loc[before_idx, 'DAYS_TO_CHURN'] = (churn_date - label_times.loc[before_idx, 
                                                                                         'CUTOFF_TIME']).dt.days
            previous_churn_date = churn_date

        else:
            label = 0

        label_times.loc[i, 'LABEL'] = label
        label_times.loc[i, 'CHURN_DATE'] = churn_date
        
        if not np.any(label_times['LABEL'] == 1):
            label_times['DAYS_TO_CHURN'] = np.nan
            label_times['CHURN_DATE'] = pd.NaT
        
    if return_act:
        return label_times.drop(columns = ['CUSTOMER_NUMBER']), activities
    
    return label_times[['CUSTOMER_NUMBER', 'CUTOFF_TIME', 'LABEL', 'DAYS_TO_CHURN', 'CHURN_DATE']].copy()

In [ ]:
df3.loc[df3.CUSTOMER_NUMBER == 0]

In [ ]:
CUSTOMER_ID = 0
customer_activities = df3.loc[df3['CUSTOMER_NUMBER'] == CUSTOMER_ID].copy()

label_times, cust_activities = label_customer(CUSTOMER_ID, customer_activities, prediction_date = 'MS', churn_days = 30, 
                                              lead_time = 1, prediction_window = 1, return_act = True)

label_times

In [ ]:
cust_activities

In [ ]:
def make_label_times(activities, prediction_date, churn_days, lead_time = 1, prediction_window = 1):
    
    label_times = []
    activities = activities.sort_values(['CUSTOMER_NUMBER', 'ACTIVITY_DATE'])
    
    for customer_id, customer_activities in activities.groupby('CUSTOMER_NUMBER'):
        lt_cust = label_customer(customer_id, customer_activities, prediction_date, churn_days, lead_time, prediction_window)
        
        label_times.append(lt_cust)
        
    return pd.concat(label_times)

In [ ]:
import time
start_time = time.time()

label_times = make_label_times(df3, prediction_date = 'MS', churn_days = 30, lead_time = 1, prediction_window = 1)

print("--- %s seconds ---" % (time.time() - start_time))

label_times

In [ ]:
cutoff_times = label_times.sort_values(by = ['CUTOFF_TIME'])
cutoff_times = cutoff_times['CUTOFF_TIME'].unique()

for ms in cutoff_times:
    distribution = label_times.loc[label_times['CUTOFF_TIME'] == ms, ['LABEL']].value_counts()
    
    if len(distribution) == 1:
        churn_rate = 0
    else:
        churn_rate = distribution[1].item()*100 / (distribution[1].item() + distribution[0].item())
    
    print('cutoff_time: ', ms, 'churn_rate: ', churn_rate)

In [ ]:
label_times['LABEL'].value_counts().plot.bar(color = 'b');
plt.xlabel('Label'); plt.ylabel('Count'); plt.title('Label Distribution with Monthly Predictions');

In [ ]:
churn_rate = label_times['LABEL'].value_counts()
churn_rate.plot(kind='pie', autopct='%1.1f%%',figsize=(10, 10))
churn_rate